In [1]:
#%pip install chatterbot-corpus -qq
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer
import NER_Attempt_02 as ner
import pickle

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/dan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/dan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
bot = ChatBot(
    'TravelJohnny',
    storage_adapter='chatterbot.storage.SQLStorageAdapter',
    database_uri='sqlite:///database.sqlite3'
)


[nltk_data] Downloading package stopwords to /home/dan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/dan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
#Bring in George's conversations...


with open('frames_list.pkl', 'rb') as f:
    frames_list = pickle.load(f)


In [4]:
# Bring in my conversations...
with open('custom_list.pkl', 'rb') as f:
    custom_list = pickle.load(f)

#Massage for trainer
#print(custom_list)

custom_list_2 =[]

for s in custom_list:
    custom_list_2.append(s[0])
    
print(custom_list_2)

['Hi there', 'How can I help you?', 'Are there any flights from Houston to San Diego', 'Yes, for which dates would you like?', 'June 3rd to June 18th', 'How much would like to spend?', 'Under 500', 'Great, Here are some options:', 'Good Morning', 'How can i help you?', 'I would like to see if I can travel from Seattle to Chicago tomorrow', 'When would you like to come back', 'In 5 weeks, but I am flexible on the day', 'What is your budget', 'Less than fifteen hundred dollars', 'Great, Here are some options:', 'Yo', 'How can I help you?', 'How much is it to go from San Diego to Los Angeles. I have to get there for a wedding.', 'When would you like to travel', 'March 1st to March 2nd', 'What is your budget', 'The cheapest you can find', 'Great, Here are some options:', 'Guten Tag', 'How can I help you?', 'I would like to fly to San Francisco on the 5th of December', 'What is your budget', 'Price is no object, and I would like first class if possible', 'Great, Here are some options:', 'He

In [ ]:
trainer = ListTrainer(bot)
trainer.train(frames_list)
trainer.train(custom_list_2)

In [9]:
print('Hello, how can I help you?')

Locations_list = []
Dates_list = []
Money_list = []

cnt = 0
cnt_return = 0
cnt_money = 0 
no_bud = False
one_way = False

while True:
    try:
        
        #Parse the customer input
        cust_input = input()        
        travel_chat_preproc = ner.word_preproc([[cust_input]])
        travel_ner_out = ner.word_ner_all(travel_chat_preproc)   
        
        #See if we found any information
        if len(travel_ner_out['Locations'])>0:
            for a in travel_ner_out['Locations']:
                Locations_list.append(a) 
        
        if len(travel_ner_out['Dates'])>0:
            for b in travel_ner_out['Dates']:
                Dates_list.append(b) 
            
        if len(travel_ner_out['Money'])>0:
            for c in travel_ner_out['Money']:
                Money_list.append(c) 
        
        print(len(Locations_list),len(Dates_list),len(Money_list))
        
        if cnt<3:
            bot_input = bot.get_response(cust_input)
            print(bot_input)
        
        else:
            
            if len(Locations_list)<2:
                print('Where do you want to travel from?')
                
            elif len(Dates_list)<2 and one_way == False:
                
                if cnt_return == 0:
                    print('When do you want to return?')
                else:
                    one_way = True
                    continue
                    
                cnt_return = cnt_return + 1
                
            elif len(Money_list)<1 and no_bud == False:
                
                
                if cnt_money == 0:
                    print('How much do you want to spend?')
                else:
                    no_bud = True
                    continue
                    
                cnt_money = cnt_money + 1
                
        
            else:
                print('Great, here are some options:')
                #Use information above to search sky scanner
                
                #Convert Dates, Location (airport codes) for usage in Flight lookup
                
                break
        
        cnt = cnt + 1
 
    except(KeyboardInterrupt, EOFError, SystemExit):
        break

Hello, how can I help you?
I'd like to go to the Bahamas
1 0 0
When would you like to travel
Monday
1 0 0
And what destination would you like to travel to?
Bahamas
2 0 0
The highest rated hotel we have in Osaka is the Aquamarine Majesty Hotel, also a 4 star hotel.
I don't want to go there
2 0 0
When do you want to return?
Saturday
2 0 0


In [6]:
# If no second loaction, ask for start location

# If no second date, assume one way

# If no money, assume cheapest

# If no dates, ask for dates, if not look at today/oneway

# If more than 3 locations -> Multicity

# Chatterbot
# https://chatterbot.readthedocs.io/en/stable/training.html


#Sky Scanner
#https://rapidapi.com/skyscanner/api/skyscanner-flight-search/endpoints


# Integration with Slack

In [7]:
bot.export_for_training('my_export.json')

AttributeError: 'ChatBot' object has no attribute 'export_for_training'

In [ ]:
import requests

url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browseroutes/v1.0/US/USD/en-US/SFO-sky/ORD-sky/2020-09-01"

querystring = {"inboundpartialdate":"2020-12-01"}

headers = {
    'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
    'x-rapidapi-key': "e9ea65cb6bmsh7a9294203a09dfep163c42jsn05f9e4a2cceb"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)